# 第3回：マルチラベル診断とAIの根拠の可視化

## 1. 目的
- 1枚の画像に複数の疾患ラベルが付与される「マルチラベル分類」を学ぶ。
- Grad-CAMを用いて、AIが画像のどこを見て判断したかをヒートマップで確認する。

In [ ]:
# --- 1. 環境セットアップとインポート ---
import sys
import os

# リポジトリをクローン（初回のみ）
if not os.path.exists('/tmp/MedMNIST-Exercise'):
    !git clone https://github.com/kshimoji8/MedMNIST-Exercise-Public.git /tmp/MedMNIST-Exercise -q

# srcディレクトリをパスに追加
sys.path.insert(0, '/tmp/MedMNIST-Exercise/src')

# exercise_logic をインポート
# （この時点では依存ライブラリをロードしないため、エラーになりません）
from medmnist_exercise import exercise_logic

# 環境初期化を実行（必要なライブラリのインストールが完了するまでここで待機します）
exercise_logic.initialize_environment()

print("✓ セットアップが完了しました。")

# 1. ChestMNISTのロード
(x_train, y_train), (x_test, y_test), info = exercise_logic.load_and_preprocess('chestmnist')

# 2. マルチラベル設定でモデル構築
model = exercise_logic.build_model(
    input_shape=(28, 28, 3), 
    num_classes=len(info['label']), 
    model_type='simple', 
    multi_label=True
)

# 3. 学習
history = model.fit(x_train, y_train, epochs=5, validation_split=0.1, batch_size=128)

In [ ]:
# 4. Grad-CAMによる可視化
img_idx = 0
target_img = x_test[img_idx:img_idx+1]

# 最後の畳み込み層を指定 (モデル構造に合わせて確認)
heatmap = exercise_logic.compute_gradcam(model, target_img, last_conv_layer_name='conv2d_2')

plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.imshow(target_img[0])
plt.title("Original X-ray")
plt.subplot(1, 2, 2)
plt.imshow(target_img[0])
plt.imshow(heatmap, cmap='jet', alpha=0.4)
plt.title("Grad-CAM Heatmap")
plt.show()